# POLI 175 - Lecture 07

## Regression (final)

## Regression and Classification

Loading packages:

In [ ]:
# If needed
using Pkg
Pkg.add("Lowess")
Pkg.add("Gadfly")
Pkg.add("RegressionTables")
Pkg.add("CovarianceMatrices")
Pkg.add("Econometrics")
Pkg.add("LinearAlgebra")
Pkg.add("MixedModelsExtras")
Pkg.update()

## Regression and Classification

Loading packages:

In [ ]:
## Loading the data
using CSV, DataFrames, Plots, GLM, StatsBase, Random
using LaTeXStrings, StatsPlots, Lowess, Gadfly, RegressionTables
using CovarianceMatrices, Econometrics, LinearAlgebra, MixedModelsExtras

# Auxiliar function
function pairplot(df)
    _, cols = size(df)
    plots = []
    for row = 1:cols, col = 1:cols
        push!(
            plots,
            scatter(
                df[:, row],
                df[:, col],
                xtickfont = font(4),
                ytickfont = font(4),
                legend = false,
            ),
        )
    end
    Plots.plot(plots..., layout = (cols, cols))
end

## Regression and Classification

Loading data:

In [ ]:
# URL of the prestige dataset
urldat = "https://raw.githubusercontent.com/umbertomig/POLI175julia/main/data/Duncan.csv"

# Load the CSV file
dat = CSV.read(download(urldat), DataFrame)

# First few obs
first(dat, 3)

## Regression

There are many packages in Julia to run Regression and Classification models. We are going to use two:

- [`GLM`](https://github.com/JuliaStats/GLM.jl) and its family (https://juliastats.org/)
- [`MLJ`](https://alan-turing-institute.github.io/MLJ.jl/dev/)

We have been using the [GLM](https://github.com/JuliaStats/GLM.jl). Next lecture we will center on the [MLJ](https://alan-turing-institute.github.io/MLJ.jl/dev/).

## Regression

### Questions

Quick reminder of a few relevant questions on the prestige of professions dataset:

Questions we answered:

- Is there a relationship between `prestige` and `income`?
- How strong is the relationship between `prestige` and `income`?
- Which variables are associated with `prestige`?
- How can we accurately predict the prestige of professions not studied in this survey?

Questions that we are still to answer:

- Is the relationship linear?
- Is there a synergy among predictors?

## Regression

### Simplest Model

In [ ]:
mod1 = lm(@formula(prestige ~ income), dat)

## Diagnostics

Several plots can help us diagnose the quality of our model.

**Warning**: Find and analyzing these violations is **more of an art**.

A careful analysis is frequent enough to ensure you have a `good` model.

## Diagnostics

### Non-linearity

When the relationship is non-linear, you could have done better using a different (more flexible) functional form.

The plot to detect this is residual in the y-axis against the fitted values in the x-axis:

![reg](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/fig5.png?raw=true)

**Plot**: Fitted Values x Raw Residuals

- Good: You should find no patterns.

- Bad: A discernible pattern tells you that you could have done better with a more flexible model.

## Diagnostics

### Non-linearity

But how to find the fitted values and the residuals in the GLM package?

In [ ]:
# Residuals
p1 = histogram(residuals(mod1))
title!("Residuals")
p2 = histogram(fitted(mod1))
title!("Fitted Values")
Plots.plot(p1, p2, layout = (1, 2))

## Diagnostics

### Non-linearity

For the `prestige` x `income` relationship:

In [ ]:
# Residual x fitted values (linearity + heteroscedasticity)
Plots.scatter(
    fitted(mod1), residuals(mod1),
    xlabel = "Fitted", ylabel = "Rediduals",
    series_annotations = text.(dat.profession, :left, :bottom, 8),
    legend = false
)

## Diagnostics

### Non-linearity

Hint: Look at the smoothing trend line (the `lowess`). You should see no discernible trend.

In [ ]:
# Residual x fitted values (linearity + heteroscedasticity)
Gadfly.plot(x=fitted(mod1), y=residuals(mod1), 
    Geom.point, Geom.smooth, Guide.xlabel("Fitted"), 
    Guide.ylabel("Residuals"))

## Diagnostics

### Non-linearity

Let's `cook` a non-linear relation:

$$ Y = 2 + X + 2 X^2 + \varepsilon $$

In [ ]:
## Cooking
Random.seed!(4321)
cooked_data = DataFrame(x = randn(100))
cooked_data.y = 2 .+ 1 .* cooked_data.x .+ 2 .* (cooked_data.x .^ 2) .+ randn(100)

## Fitting (wrong)
mod2 = lm(@formula(y ~ x), cooked_data)

## Diagnostics

### Non-linearity

The smoothing trend line (the `lowess`) show a discernible trend.

In [ ]:
# Residual x fitted values (linearity + heteroscedasticity)
Gadfly.plot(x=fitted(mod2), y=residuals(mod2), 
    Geom.point, Geom.smooth, Guide.xlabel("Fitted"), 
    Guide.ylabel("Residuals"))

## Diagnostics

### Non-linearity

Let us fit the *right* model now: $$ Y = 2 + X + 2 X^2 + \varepsilon $$

In [ ]:
mod3 = lm(@formula(y ~ x + exp(x)), cooked_data)

## Diagnostics

### Non-linearity

And the residuals versus fitted values look better when fitting the correctly specified model:

In [ ]:
# Residual x fitted values (linearity + heteroscedasticity)
Gadfly.plot(x=fitted(mod3), y=residuals(mod3), 
    Geom.point, Geom.smooth, Guide.xlabel("Fitted"), 
    Guide.ylabel("Residuals"))

## Diagnostics

### Non-linearity

There exist a test called [Ramsey RESET test](https://en.wikipedia.org/wiki/Ramsey_RESET_test).

I strongly suggest you not to use these, since it usually does not identify better relationships than polynomial, while non-linearity can be something extremely complex.

We will learn how to deal with this in a few lectures.

<div class="cite2c-biblio"></div>## Diagnostics

### Heteroscedasticity

It is fancy wording to say that the variance in error is not constant.

It usually means that you are better at fitting some range of the predictors than others.

![reg](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/fig7.png?raw=true)

**Plot:** Fitted Values x Residuals

- Bad (left-hand side): A funnel-shaped figure tells you that you may have heteroscedasticity. It invalidates simple standard errors assumptions.

- Good (right-hand side): You should find no patterns.

## Diagnostics

### Heteroscedasticity

For the `prestige` x `income` relationship:

In [ ]:
# Residual x fitted values (linearity + heteroscedasticity)
Gadfly.plot(
    x = fitted(mod1), y = residuals(mod1),
    label = dat.profession,
    Geom.point, 
    Geom.smooth,
    Geom.label,
    Guide.xlabel("Fitted"), 
    Guide.ylabel("Residuals"),
    Guide.title("Fitted versus raw residual plot"))

## Diagnostics

### Heteroskedasticity

Let us `cook` a heteroskedastic model:

$$ Y = 2 + 3 X + \tilde{\varepsilon} $$

Where $\text{Cov}[\tilde{\varepsilon}] \ \neq \ \sigma^2I$.

In this particular case, let us make the variance of the residuals to look like a football:

In [ ]:
## Cooking (Het-error term)
Random.seed!(4321)
cooked_data = DataFrame(x = randn(100))
cooked_data.y = 2 .+ 3 .* cooked_data.x .+ (maximum(cooked_data.x .^ 2 .+ 1) .- ((cooked_data.x).^2)) .* randn(100);

In [ ]:
## Fitting
mod4 = lm(@formula(y ~ x), cooked_data)

## Diagnostics

### Heteroscedasticity

For the cooked data:

In [ ]:
# Residual x fitted values (linearity + heteroscedasticity)
Gadfly.plot(x=fitted(mod4), y=residuals(mod4), 
    Geom.point, Geom.smooth, Guide.xlabel("Fitted"), 
    Guide.ylabel("Residuals"))

## Diagnostics

### Heteroscedasticity

Our standard errors are wrong. So are our:

1. Confidence Intervals
1. P-values
1. T-stats

In these cases, we need **robust standard errors**:

In [ ]:
# Standard Errors for the heteroskedastic model
CovarianceMatrices.stderror(mod4)

In [ ]:
# Corrected Standard Errors for the heteroskedastic model
# Note: several types of corrections...
CovarianceMatrices.stderror(CovarianceMatrices.HC1(), mod4)

### Outliers

- Outliers are values very far away from most values predicted by the model.

- Sometimes, it is correct, but frequently it may tell you that you made a mistake in collecting the data!

![reg](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/fig8.png?raw=true)

**Plot**: Fitted x Studentized residuals (right-hand side plot)

- Best: You should find no extreme values in the plot.

- Bad: An extreme value can affect your RSE, $R^2$, and mess up with p-values.

## Diagnostics

### Outliers

A few important measurements (all vectors, computed for each data point):

[**Studentized residual**](https://en.wikipedia.org/wiki/Studentized_residual): Residual weighted by the leverage of the point. Studentized because it follows the [Student's t distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution).

[**Leverage**](https://en.wikipedia.org/wiki/Leverage_(statistics)): It represents how far away a given *target* value is compared to other *target* values $\bigg(\dfrac{\partial \hat{y}_i}{\partial y_i}\bigg)$.

[**Cook's Distance**](https://en.wikipedia.org/wiki/Cook%27s_distance): It measures how much the regression model changes when we remove a given observation.

In [ ]:
#= We need to compute the studentized residual
   (https://en.wikipedia.org/wiki/Studentized_residual)
   I wrote this function to facilitate the work
=#
function lm_measures(lmod)
    X = modelmatrix(lmod)
    RSS = sum((residuals(lmod)).^2)
    sigma_hat = sqrt(RSS/dof_residual(lmod))
    leverage = diag(X * inv(transpose(X) * X) * transpose(X))
    studentized_resid = residuals(lmod) ./ (sigma_hat .* sqrt.(1 .- leverage))
    return leverage, studentized_resid, cooksdistance(lmod)
end

## Diagnostics

### Outliers

In [ ]:
Gadfly.plot(
    x=fitted(mod1), y=lm_measures(mod1)[2], 
    Geom.point, label = dat.profession, 
    Guide.xlabel("Fitted"),
    Guide.ylabel("Studentized Residuals"), 
    Geom.label,
    yintercept = [-2.0, 2.0], 
    Geom.hline()
)

## Diagnostics

### Outliers

In [ ]:
show(dat, allrows = true, allcols = true)

## Diagnostics

### Outliers

In [ ]:
mod1

In [ ]:
mod2 = lm(@formula(prestige ~ income), 
    dat[(dat.profession .!= "minister"), :])


In [ ]:
mod3 = lm(@formula(prestige ~ income), 
    dat[(dat.profession .!= "conductor"), :])

In [ ]:
mod4 = lm(@formula(prestige ~ income), 
    dat[(dat.profession .!= "minister") .& (dat.profession .!= "conductor"), :])

## Diagnostics

### High Leverage

Having a very unusual value, that could potentially tilt the regression line towards it

***If high leverage and outlier, bad combination!***

![reg](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/fig9.png?raw=true)

**Plot**: Leverage x Studentizided residuals

- Best: You should find no extreme values in the plot.

- Bad: An extreme value can affect your fit.

In [ ]:
Gadfly.plot(
    x=lm_measures(mod1)[1], y=lm_measures(mod1)[2], 
    Geom.point, label = dat.profession, 
    Guide.xlabel("Leverage"),
    Guide.ylabel("Studentized Residuals"), 
    Geom.label,
    yintercept = [-2.0, 2.0], 
    Geom.hline()
)

## Diagnostics

### Cook's Distance

Measures whether removing a given observation tilts the regression coefficients.

**Plot**: Cook's Distance

- Best: You should find no values in the farther diagonal

- Bad: Extreme value in the farther diagonal represents data that is highly influential (high leverage) and high changes in coefficients (high Cook's D)

In [ ]:
Gadfly.plot(
    y = dat.profession, x = lm_measures(mod1)[3], 
    Geom.point
)

## Diagnostics

When we run multiple linear regression, we add *multicollinearity* to the diagnostics we have seen so far.

### Multicollinearity

Multicollinearity is when your predictors are highly correlated. In extreme cases, it messes up with the standard errors in our model (problems with inverse matrix).

In [ ]:
## Pairplot to check
println(first(dat, 1))
pairplot(dat)

## Diagnostics

### Multicollinearity

One measure of multicollinearity is the [*Variance Inflation Factor*](https://en.wikipedia.org/wiki/Variance_inflation_factor).

How much the multicollinearity is messing up with the estimates.
    
To compute, it is fairly easy. As a rule-of-thumb, we would like to see values lower than 5.

***It is rarely a problem, though... Especially with large datasets.***

In [ ]:
modfull = lm(@formula(prestige ~ income + education + type), dat)
println(modfull)
MixedModelsExtras.vif(modfull)

## Diagnostics

Rules to diagnostics:

1. Always assume heteroskedasticity (use robust standard errors)
1. Check for outliers
1. Drop a few observations and rerun the regression. Do that with most (all) observations.
1. Graph your residuals.
1. If you have extreme values, make sure your results remain valid after dropping them.

In a nutshell, pay attention to what you are doing!

# Model Selection

## Linear Model Selection

- We usually have a large set of predictors that could be used.
    + Which predictors to pick becomes a task.

- If we are trying to interpret things and learn from the data, then which predictors are correlated with the outcome is informative:
    + Again, picking predictors becomes a task.
    
- In this and the following lecture, we will learn how to do that systematically. 

## Linear Model Selection

### Subset Selection

- In here, we are going to consider techniques to select a subset of predictors based on a performance metric.

## Linear Model Selection

### Subset Selection

#### Best Subset Selection

**Algorithm:**

1. Let $M_0$ denote the null model, which contains no predictors. This model predicts the sample mean for each observation.

2. For $k = \{1, 2, \cdots, p\}$:
    1. Fit all $p \choose k$ models with exactly $k$ predictors.
    2. Pick the *best* among these models and call it your $M_k$

3. Select a single best model from among $M_0, \cdots, M_p$ using cross-validated prediction error, $C_p$, AIC, BIC, or adjusted $R^2$.

## Linear Model Selection

### Subset Selection

#### Best Subset Selection

![img ms1](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/ms1.png?raw=true)

## Linear Model Selection
### Subset Selection

#### Best Subset Selection

Notes:

1. You can do this with Logistic Regression: change RSS with [*deviance*](https://en.wikipedia.org/wiki/Deviance_(statistics)).
    + In this case, our friend $- 2\ln({\hat {L}})$ does very well!

2. Best Selection is excellent but fits around $2^p$ models.
    + $p = 10$ means around 1000 estimates.

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Forward Selection

**Algorithm**:

1. Let $M_0$ denote the null model, which contains no predictors.

2. For $k = \{0, 1, 2, \cdots, p-1\}$:
    1. Consider all $p - k$ models that augments $M_k$ by one predictor.
    2. Pick the *best* among these $p-k$ models, and call it your $M_{k+1}$.

3. Select a single best model from among $M_0, \cdots, M_p$ using cross-validated prediction error, $C_p$, AIC, BIC, or adjusted $R^2$.

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Forward Selection

- Much more efficient:
    + It fits a total of $1 + \dfrac{p(p+1)}{2}$ models.
    + If $p = 20$, the Best Selection would fit 1,048,576
    + If $p = 20$, the Forward Step Selection would fit 211 models.

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Forward Selection

- The catch: It is not guaranteed that it is going to find the *best subset* model.

- Example: Let $p = 3$.
    + Suppose that the best model involves $v2$ and $v3$.
    + But suppose that within the models with only one variable, $v1$ would do better.
    + Then, Forward Step Selection would never pick this model!

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Forward Selection

![img ms2](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/ms2.png?raw=true)

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Backward Selection

**Algorithm**:

1. Let $M_p$ denote the *full* model, which contains all $p$ predictors.

2. For $k = \{p, p-1, p-2, \cdots, 1\}$:
    1. Consider all $k$ models that contail all but one predictor in $M_k$, for a total of $k-1$ predictors.
    2. Pick the *best* among these $k$ models, and call it your $M_{k-1}$.

3. Select a single best model from among $M_0, \cdots, M_p$ using cross-validated prediction error, $C_p$, AIC, BIC, or adjusted $R^2$.

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Backward Selection

- Computational Efficiency:
    + It fits a total of $1 + \dfrac{p(p+1)}{2}$ models.
    + Same efficiency as the Forward Selection.

- Catch:
    + Same catch as the Forward Selection: It does not guarantee the pick of the best model.

## Linear Model Selection

### Subset Selection

#### Stepwise Selection: Hybrid Approaches

- Combinations of *Forward* and *Backwards* that intend to mimic the *Best Selection*.

- Many available.

- But the trade-offs are clear: 
    + Computational efficiency
    + Likelihood of picking the best model

## Linear Model Selection

### Subset Selection

#### What is Best?

- For each of the $M_k$ models, this is it:
    + RSS: Residual Sum of Squares: We want it to be the lowest possible.
    + $R^2$: We want it to be the highest possible.
    + And for Logistic or other GLM Regressions, *deviance*.

## Linear Model Selection

### Subset Selection

#### What is Best?

- Catches: 
    1. RSS and $R^2$ always improve with more variables.
    2. We want to look at the *testing set goodness-of-fit*, not the *training sets goodness-of-fit*!

- And that is why RSS and $R^2$ are not used in Step 3:
    - We need something that eventually gets worse the more variables we throw in.

## Linear Model Selection

### Subset Selection

#### What is Best?

**Important:**

- Training Set MSE generally underestimates the testing set MSE.

$$ \text{MSE} \ = \ \dfrac{RSS}{n} $$

- But before, we could not split data into *training* and *testing*. 
    + This is a more recent feature, thanks to our increased computational power.

- Here are a few stats that we can fit in the training set.

## Linear Model Selection

### Subset Selection

#### What is Best?

- [$C_P$](https://en.wikipedia.org/wiki/Mallows%27s_Cp) in a model containing:
    - $d$ predictors.
    - $n$ observations.
    - $\widehat{\sigma}^2$ the variance of the error in the full model with all predictors.

$$ C_p \ = \ \dfrac{1}{n}(RSS + 2\times d \times \widehat{\sigma}^2) $$

- The smaller, the better.

## Linear Model Selection

### Subset Selection

#### What is Best?

- [AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion) or $C_p$: AIC is a goodness-of-fit parameters that goes lower when you are improving the model
    + But for every variable you add, it penalizes it.
    + If by adding more variables, it goes up, then your model is getting more complex without adding much.
    
$$ \text{AIC} \ = \ 2d - 2\log({\hat {L}}) $$

## Linear Model Selection

### Subset Selection

#### What is Best?

- The maximum likelihood and least squares are the same for models with Gaussian errors.

$$ \text{AIC} \ = \ \dfrac{1}{n} (RSS + 2\times d \times \widehat{\sigma}^2) $$

## Linear Model Selection

### Subset Selection

#### What is Best?

- [BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion) is another goodness-of-fit, but this one penalizes the addition of variables at a higher rate.

$$ \text{BIC} = k\log(n) - 2\log({\widehat {L}}) $$

- Note the difference from the AIC: instead of multiplying by 2, it is multiplying by $\ln(n)$!

- Again, lower values are better.

## Linear Model Selection

### Subset Selection

#### What is Best?

- Or if you want the one with the least square errors:

$$ \text{AIC} \ = \ \dfrac{1}{n} (RSS + \log(n) \times d \times \widehat{\sigma}^2)  $$ 

## Linear Model Selection

### Subset Selection

#### What is Best?

- [Adjusted $R^2$](https://en.wikipedia.org/wiki/Bayesian_information_criterion): It is a change in the $R^2$ to penalize the addition of regressors.

$$ \overline{R}^{2} = 1-(1-R^{2})\dfrac{n-1}{n-d} \ = \ 1 - \dfrac{\frac{RSS}{n - d - 1}}{\frac{TSS}{n - 1}}$$

- $R^2$ always increase but the $\overline{R}^2$ may increase or decrese.

- **Not like the others:** This one, the higher, the better.

## Linear Model Selection

### Subset Selection

#### What is Best?

![img ms3](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/ms3.png?raw=true)

## Linear Model Selection

### Subset Selection

#### What is Best?

- $C_P$, AIC, and BIC all have a solid theoretical justification.
    + Many of them have something we call Large Sample Properties.
    + Check their Wikipedia of them. They converge to nice, important values.

- $\overline{R}^{2}$ does not.

## Linear Model Selection

### Subset Selection

#### What is best? Validation and Cross-Validation

- The main advantage is obvious: You look at testing errors!

- The other main advantage is regarding estimating parameters:
    + $C_P$, AIC, and BIC all have a strong theoretical justification, which is reassuring.
    + But sometimes, one does not know the theory behind an estimate to compute statistics or even standard errors.
    + E.g., which $\widehat{\sigma}^2$ should we pick?
    + Validation and Cross-Validation do great in these cases!

## Linear Model Selection

### Subset Selection

#### What is best? Validation and Cross-Validation

![img ms4](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/ms4.png?raw=true)

# Questions?

# See you next class
